# Kernel ridge regression (KRR)

Regularization is especially important in non-parametric models. In order to avoid over-fitting the model, penalizing models that change very sharply is necessary. By adding a penalty for very large parameters in the loss function, the error of the model is kept small while the size of the parameters is also kept small. By using a kernel and regularize on the sum of squared parameters it is called Kernel Ridge Regression, or KRR. In this project, we use **radial basis function** as the kernel in KRR. The loss function for KRR is as follows:

$$\mathcal{L} = \sum_i \varepsilon_i^2 + \alpha ||\vec{w}||_2$$

## Hyperparameter tuning strategy

There are two main hyperparameters in the KRR:
1. $\alpha$, which controls the strength of regularization
2. $\gamma$ in the RBF:
$$\operatorname{rbf}(x_i, x_j) = \exp (-\gamma (x_i - x_j)^2),$$ 
or 
$$\gamma = \frac{1}{2\sigma^2}$$, where $\sigma$ is the width of the gaussian function:
$G(x_i) = \exp\left(\frac{-(x_i - \mu)^2}{2\sigma^2}\right)$.

When the hyperparameters are tuned, gammas of different orders of magnitude are looped through. `GridSearchCV` is then used to find out the alpha of different orders of magnitude which produces the highest r square value for a given gamma. 
`GridSearchCV` is called two times to refine the hyperparameters. In the second round of tuning, the range of alpha and gamma is limited to 0.5~5\*optimal value in the first round.

## Regression using unscaled data

In [ ]:
sigmas = np.array([1E-4, 5E-4, 1E-3, 5E-3, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 15, 20, 25, 30, 40, 50, 60])
gammas = 1./(2*sigmas**2)
alphas = np.array([1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100])
parameter_ranges = {'alpha':alphas}

KRR_r2_test_list = []
KRR_coef_list = []
KRR_gamma_list = []
KRR_alpha_list = []

for gamma in gammas:
    KRR = KernelRidge(kernel='rbf',gamma=gamma)
    KRR_search = GridSearchCV(KRR, parameter_ranges, cv=kf)
    KRR_search.fit(X_train_test, y_train_test)
    KRR_r2_test_list.append(KRR_search.best_score_)
    KRR_gamma_list.append(KRR_search.best_estimator_.gamma)
    KRR_alpha_list.append(KRR_search.best_estimator_.alpha)
    KRR_coef_list.append(KRR_search.best_estimator_.dual_coef_)
    
best_i = np.array(KRR_r2_test_list).argmax()
KRR_r2_best_1= KRR_r2_test_list[best_i]
KRR_gamma_best_1 = KRR_gamma_list[best_i]
KRR_alpha_best_1 = KRR_alpha_list[best_i]
KRR_coef_best_1 = KRR_coef_list[best_i]

print("Highest r2={:.3f},\toptimal gamma={:.0e},\toptimal alpha={:.0e}".format(KRR_r2_test_list[best_i], KRR_gamma_best_1, KRR_alpha_best_1))

In the first round of the hyperparameter tuning, r2 score is high, indicating good performance of the model. 

In [ ]:
#Second interation with smaller range of alphas and gammas
gammas = np.array([KRR_gamma_best_1*0.5,KRR_gamma_best_1*0.75,KRR_gamma_best_1,KRR_gamma_best_1*2.5,KRR_gamma_best_1*5])
alphas = np.array([KRR_alpha_best_1*0.5,KRR_alpha_best_1*0.75,KRR_alpha_best_1,KRR_alpha_best_1*2.5,KRR_alpha_best_1*5])
parameter_ranges = {'alpha':alphas}

KRR_r2_test_list = []
KRR_coef_list = []
KRR_gamma_list = []
KRR_alpha_list = []
    
for gamma in gammas:
    KRR = KernelRidge(kernel='rbf',gamma=gamma)
    KRR_search = GridSearchCV(KRR, parameter_ranges, cv=kf)
    KRR_search.fit(X_train_test, y_train_test)
    KRR_r2_test_list.append(KRR_search.best_score_)
    KRR_gamma_list.append(KRR_search.best_estimator_.gamma)
    KRR_alpha_list.append(KRR_search.best_estimator_.alpha)
    KRR_coef_list.append(KRR_search.best_estimator_.dual_coef_) 
    
best_i = np.array(KRR_r2_test_list).argmax()
KRR_r2_best_2= KRR_r2_test_list[best_i]
KRR_gamma_best = KRR_gamma_list[best_i]
KRR_alpha_best = KRR_alpha_list[best_i]
KRR_coef_best = KRR_coef_list[best_i]

print("Highest r2={:.3f},\toptimal gamma={:.0e},\toptimal alpha={:.0e}".format(KRR_r2_test_list[best_i], KRR_gamma_best, KRR_alpha_best))

In the second round of the hyperparameter tuning, the optimal values of gamma and alpha are different. We don't see a significant improvement in r2 scores, but in the first round the models are already well-performed.

### Regression using scaled data with hyperparameter tuning

In [ ]:
#Regression with scaled data
#GridSearchCV alpha iterate gamma

sigmas = np.array([1E-4, 5E-4, 1E-3, 5E-3, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 15, 20, 25, 30, 40, 50, 60])
gammas = 1./(2*sigmas**2)
alphas = np.array([1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100])
parameter_ranges = {'alpha':alphas}

KRR_r2_test_scaled_list = []
KRR_coef_scaled_list = []
KRR_gamma_scaled_list = []
KRR_alpha_scaled_list = []

for gamma in gammas:
    KRR = KernelRidge(kernel='rbf',gamma=gamma)
    KRR_search = GridSearchCV(KRR, parameter_ranges, cv=kf)
    KRR_search.fit(X_train_test_scaled, y_train_test)
    KRR_r2_test_scaled_list.append(KRR_search.best_score_)
    KRR_gamma_scaled_list.append(KRR_search.best_estimator_.gamma)
    KRR_alpha_scaled_list.append(KRR_search.best_estimator_.alpha)
    KRR_coef_scaled_list.append(KRR_search.best_estimator_.dual_coef_)    

    
best_i = np.array(KRR_r2_test_scaled_list).argmax()
KRR_r2_best_scaled1= KRR_r2_test_scaled_list[best_i]
KRR_gamma_best_scaled_1 = KRR_gamma_scaled_list[best_i]
KRR_alpha_best_scaled_1 = KRR_alpha_scaled_list[best_i]
KRR_coef_best_scaled_1 = KRR_coef_scaled_list[best_i]

print("Highest r2={:.3f},\toptimal gamma={:.0e},\toptimal alpha={:.0e}".format(KRR_r2_test_scaled_list[best_i], KRR_gamma_best_scaled_1, KRR_alpha_best_scaled_1))


In [ ]:
#second interation with smaller range of alphas and gammas

gammas = np.array([KRR_gamma_best_scaled_1*0.5,KRR_gamma_best_scaled_1*0.75,KRR_gamma_best_scaled_1,KRR_gamma_best_scaled_1*2.5,KRR_gamma_best_scaled_1*5])
alphas = np.array([KRR_alpha_best_scaled_1*0.5,KRR_alpha_best_scaled_1*0.75,KRR_alpha_best_scaled_1,KRR_alpha_best_scaled_1*2.5,KRR_alpha_best_scaled_1*5])
parameter_ranges = {'alpha':alphas}

KRR_r2_test_scaled_list = []
KRR_coef_scaled_list = []
KRR_gamma_scaled_list = []
KRR_alpha_scaled_list = []

for gamma in gammas:
    KRR = KernelRidge(kernel='rbf',gamma=gamma)
    KRR_search = GridSearchCV(KRR, parameter_ranges, cv=kf)
    KRR_search.fit(X_train_test_scaled, y_train_test)
    KRR_r2_test_scaled_list.append(KRR_search.best_score_)
    KRR_gamma_scaled_list.append(KRR_search.best_estimator_.gamma)
    KRR_alpha_scaled_list.append(KRR_search.best_estimator_.alpha)
    KRR_coef_scaled_list.append(KRR_search.best_estimator_.dual_coef_)    

    
best_i = np.array(KRR_r2_test_scaled_list).argmax()
KRR_r2_best_scaled2= KRR_r2_test_scaled_list[best_i]
KRR_gamma_best_scaled = KRR_gamma_scaled_list[best_i]
KRR_alpha_best_scaled = KRR_alpha_scaled_list[best_i]
KRR_coef_best_scaled = KRR_coef_scaled_list[best_i]

print("Highest r2={:.3f},\toptimal gamma={:.0e},\toptimal alpha={:.0e}".format(KRR_r2_test_scaled_list[best_i], KRR_gamma_best_scaled, KRR_alpha_best_scaled))

After the search range of the hyperparameters is refined, the optimal values change a little bit，but the r2 remains nearly the same.

To avoid overfitting, smoother models are preferred. We look at how using different input data (unscaled and scaled) affects the size and distribution of the coefficients in KRR models. Histograms are drawn and the largest absolute values of the coefficients in KRR using unscaled and scaled data are printed below.

In [ ]:
print('The model using unscaled data has {} coefficients.'.format(len(KRR_coef_best)))

fig, ax = plt.subplots()
ax.hist(KRR_coef_best)
ax.set_xlabel('Coefficient Value')
ax.set_ylabel('Number of Coefficients')
print('The largest coefficient is {:.3f}.'.format(max(abs(KRR_coef_best))[0]));
KRR_largest_coef=max(abs(KRR_coef_best))[0]

In [ ]:
print('The model using scaled data has {} coefficients.'.format(len(KRR_coef_best_scaled)))

fig, ax = plt.subplots()
ax.hist(KRR_coef_best_scaled)
ax.set_xlabel('Coefficient Value')
ax.set_ylabel('Number of Coefficients')
print('The largest coefficient is {:.3f}.'.format(max(abs(KRR_coef_best_scaled))[0]));
KRR_largest_coef_scaled=max(abs(KRR_coef_best_scaled))[0]

The coefficients are normally distributed for two models. The values of coefficients in KRR models, which include regularization terms in the loss function, are in a reasonable range. One thing to notice is that the size of the coefficients is one order of magnitude smaller after scaling the data by comparing the largest coefficient in the model. This indicates that KRR using scaled data is more likely to be not overfitted with high accuracy.